In [1]:
import requests
import pandas as pd

import re

from bs4 import BeautifulSoup

In [2]:
# site data
URL = "https://kreyol.com/dictionary/"
pages = ["Aa.html", "Bb.html", "Cc.html", "Dd.html", "Ee.html", 
        "Ff.html", "Gg.html", "Hh.html", "Ii.html", "Jj.html", 
        "Kk.html","Ll.html", "Mm.html", "Nn.html", "Oo.html",
        "Pp.html", "Qq.html", "Rr.html", "Ss.html", "Tt.html",
        "Uu.html", "Vv.html", "Ww.html", "Xx.html", "Yy.html",
        "Zz.html"]

# Testing

In [3]:
x = URL + pages[0]
print(x)
page = requests.get(URL + "Aa.html")
# print(page.text)

soup = BeautifulSoup(page.content, "html.parser")

y = soup.find_all("p")
# print(y)
words = []
count = 0
for element in y:
    # print(type(element))
    # print(type(element.text))
    if "/" in element.text and "href" not in element.text:
#         print(count, "-", element)
        words.append(element.text)
    count += 1
    
print(len(words))
# print(words)



https://kreyol.com/dictionary/Aa.html
3


In [4]:
# a = words[0].split("\n")
# print(a[0] == "")
# print(a)
test = words[0]
found = re.search("\([^\(\)]*\n[^\(]*\)",test)

while found:
    start, end = found.span()
    print(start, ", ", end)
    print(test[start:end])
    ran = test[start:end]
    ind = ran.index("\n")
    print(ind)
    temp = list(test)
    print("character at start + ind:", temp[start + ind], "neat!")
    temp[start + ind] = ""
#     print(temp[end-2])
    test = "".join(temp)
    found = re.search("\([^\(\)]*\n[^\(]\)",test)
        
a = test.split("\n")
        
wordlist_test = pd.DataFrame(columns=["word", "creole_word", "creole name"])
flip = False

for entry in a:
    if entry != "":  # check if the itme isn't an empty string
        if not flip and "Kreyol / English" in entry: # after this page on the site the side the english is on and the side the creole is on flips
            flip = True
            continue
        if "/" in entry:
            text = ["", "", "hatian"]
            if not flip:
                temp = entry.split("/")
#                 print(temp)
                text[0] = temp[0].strip()
                text[1] = temp[1].strip()
            else:
                temp = entry.split("/")
#                 print(temp)
                text[0] = temp[1].strip()
                text[1] = temp[0].strip()

            wordlist_test.loc[len(wordlist_test.index)] = text

377 ,  423
(loathe, to regard with extreme 
  repugnance)
32
character at start + ind: 
 neat!


In [5]:
wordlist_test

,word,creole_word,creole name
0,"a, an",youn,hatian
1,aback,pa sipriz (taken by surprise),hatian
2,abandon,"abandone, kite",hatian
3,abase,"abese (to lower in rank-office-prestige), ~dim...",hatian
4,abash,demoralize (to destroy the self-confidence of),hatian
...,...,...,...
738,Haiti,Ayiti (Peyi Dayiti),hatian
739,Souteast Asia,Azi di Disès,hatian
740,East Asia,Azi de Lès,hatian
741,haven,azil,hatian


In [6]:
wordlist_test.loc[9, "creole_word"]

'deteste (loathe, to regard with extreme   repugnance), rayi (hate)'

# Parsing Program

In [28]:
"""
name: split_words
param: word_str(string) - a string that contains the creole word(s) from the site that we may have to split into a list of words
return: the list of words present inthe word_str parameter

Description:
Part of the site has words formatted like this: deteste (loathe, to regard with extreme repugnance), rayi (hate). The problem if we try
to split on the commas is that the comma after loathe will be split too, causing the list to be incorrect and for information to be cut off.
So, if we come across cases like this, then we need to split the string in the way that ignores those internal commas.
"""

def split_words(word_str):
    hold = word_str
    
    found = re.search("\([^\(\)]*,[^\(]*\)",hold)

    if found: # if we found an internal comma
        print("here")
        while found: # lcheck the string until there are no internal commas left
            print(found.string)
#             print(type(found))
            start, end = found.span()
            ind = hold[start:end].find(",") # get the index where the internal comma is located
#             print(hold[start:end])
            print(ind)
            if ind == -1:
                break
            
            temp = list(hold)  # make a temporary list of the characters in the string
            temp[start + ind] = "%" # replace the internal commas with %

            hold = "".join(temp) # join the list back into a string
            found = re.search("\([^\(\)]*,[^\(]*\)",hold) # check the string again for more internal commas

        words = test.split(",") # now that all the internal commas are gone, separate the words
        for ii in range(len(words)): # go through the list of words/senteces and put the internal commas back
            if "%" in words[ii]:
                words[ii] = words[ii].replace("%", ",")
            words[ii] = words[ii].strip()
    else: #if there is no internal comma, then just return the split of the word string on the commas
        words = word_str.split(",")
                
    return words

In [29]:
wordlist = pd.DataFrame(columns=["word", "creole_word", "creole name"]) # the dataframe we will be storing the word data in


for site_page in pages: # loop through all the alphabet pages on the site
    x = URL + site_page
    print(x) # print the URL of the page that we are currently looking at
    
    
    page = requests.get(x) # get the HTML data for that page
    soup = BeautifulSoup(page.content, "html.parser") # pull the data from the site
    y = soup.find_all("p") # only get the things in the <p> tags
    
    # clean the data in the <p> tags so we only get the word data
    word_sections = []
    count = 0
    
    # go through all the elements soup got and only save those elements that have a slash - the reason that all definitions have slashes in them 
    for element in y:
        if "/" in element.text and "href" not in element.text:
            word_sections.append(element.text)
        count += 1

    
    words_split = [] # the master list of word strings (the sentences from the site with the english and creole words)

    # loop through all the word data and chunks and separate them out into the individual word lines
    for chunk in word_sections:
        
        curr_chunk = chunk
        
        """
        There are certain parts of the page that are word wrapped on the site. however when you parse the html these word wraps are done
        using newline characters (usually if they have paranethis explaining the words). This causes a problem when we use the newline 
        character to split and separate the words. So we have to go through each word chunk that has these newline word wraps and change
        them to an empty string.
        """
        found = re.search("\([^\(\)]*\n[^\(]*\)",curr_chunk) # find a plae where there is a newline wrap

        while found: # loop until you have replaced all the newline word wrapping in the current word chunk with white spaces
            start, end = found.span() # get the range the area that the newline wrapping was found
#             print(start, ", ", end)
#             print(test[start:end])
#             ran = curr_chunk[start:end]
            ind = curr_chunk[start:end].index("\n") # within this range, find the index that the newline character is located
#             print(ind)
            temp = list(curr_chunk) # split the string into a list so we can change the characters
#             print("character at start + ind:", temp[start + ind], "neat!")
            temp[start + ind] = "" # access the index that we found and replace what is there with an empty string
        #     print(temp[end-2])
            curr_chunk = "".join(temp) # put the chunk pack together into a string
            found = re.search("\([^\(\)]*\n[^\(]\)",curr_chunk) # check the chunk again to see if it has any more newline wrapping
        
        z = curr_chunk.split("\n") # split the chucnk of words by the remaining newline characters, creating a list of strings
        words_split.extend(z) # append the list of strings to the master list of word strings

#     print(words_split)
    """
    'flip' keeps track of which part of the page that we are looking at. Part of the pages keep the english words on the right of the / and the
    creole words on the left. However, part of the way through some of these pages the words flip, putting the creole words on the right
    and the english words on the left
    """   
    flip = False
        
    for entry in z: # loop through every entry that was found
        if entry != "":  # check if the itme isn't an empty string
            if not flip and ("Kreyol / English" in entry or "kreyol - english" in entry): # after this page on the site the side the english is on and the side the creole is on flips
                flip = True
                continue
            if "/" in entry and "Kreyol pronunciation" != entry:
                text = ["", "", "hatian"]
                temp = entry.split("/") # separate the enlish and creole words
                
                if not flip: # check to see which side of the / the words are on
                    text[0] = temp[0].strip() # english words
                    text[1] = split_words(temp[1].strip()) # Creole words
                                       
                else:
                    text[0] = temp[1].strip() # English words                   
                    text[1] = split_words(temp[0].strip()) # Creole
                    
                # get rid of the parts of speech indicators from the english words
                if "(v)" in text[0]:
                    text[0] = text[0].replace("(v)", "")
                    text[0] = text[0].strip()

                if "(n)" in text[0]:
                    text[0] = text[0].replace("(n)", "")
                    text[0] = text[0].strip()

                if "(adj)" in text[0]:
                    text[0] = text[0].replace("(adj)", "")

                
                wordlist.loc[len(wordlist.index)] = text
#             elif "/" not in entry and "Kreyol pronunciation" != entry:
#                 wordlist.iloc[len(wordlist.index)-1, 1].append(entry)


https://kreyol.com/dictionary/Aa.html
here
ensite (incite, urge on, instigate), eksite (excite)
7
ensite (incite% urge on, instigate), eksite (excite)
16
here
deteste (loathe, to regard with extreme   repugnance), rayi (hate)
7
here
pouri (rotten, lousy)
7
here
a la fen (alè ki le, finalman)
10
here
anbete (traka, nwi)
6
here
andedan (a, an)
2
here
anfen (a la fen, alè ki le, finalman)
9
anfen (a la fen% alè ki le, finalman)
20
here
animal (zannimo, bèt, animo)
8
animal (zannimo% bèt, animo)
13
here
anka (ka galon, ka)
9
here
anlè (anro, soutèt)
5
here
anmède (anbete, traka, nwi)
7
anmède (anbete% traka, nwi)
14
here
anniye (anmède, anbete)
7
here
anpil (trè, bien)
4
here
ant (nan mitan, omilye)
10
here
anyèl (chak ane, lan)
9
here
ase (gran, laj , sifizan)
5
ase (gran% laj , sifizan)
11
here
ase (anverite, tou)
9
here
aswè (sware, swa)
6
here
atak (peryòd, kriz)
7
here
atenn (rive, jwenn)
5
here
avrip (brid sou kou, bripbrip)
13
https://kreyol.com/dictionary/Bb.html
https://kreyol.com

In [30]:
wordlist

,word,creole_word,creole name
0,"a, an",[youn],hatian
1,aback,[pa sipriz (taken by surprise)],hatian
2,abandon,"[abandone, kite]",hatian
3,abase,"[abese (to lower in rank-office-prestige), ~d...",hatian
4,abash,[demoralize (to destroy the self-confidence of)],hatian
...,...,...,...
5817,tool,[zouti],hatian
5818,oyster,[zuit (zwit)],hatian
5819,goose,[zwa],hatian
5820,bird,[zwazo],hatian


In [31]:
wordlist.head(10)

,word,creole_word,creole name
0,"a, an",[youn],hatian
1,aback,[pa sipriz (taken by surprise)],hatian
2,abandon,"[abandone, kite]",hatian
3,abase,"[abese (to lower in rank-office-prestige), ~d...",hatian
4,abash,[demoralize (to destroy the self-confidence of)],hatian
5,abbreviate,[abreje],hatian
6,abdicate,[abdike],hatian
7,abduct,"[kidnape, anlve]",hatian
8,abet,"[A\n\n a, an / youn\n aback / pa sipriz (tak...",hatian
9,abhor,"[A\n\n a, an / youn\n aback / pa sipriz (tak...",hatian


In [32]:
wordlist.loc[9, "creole_word"]

['A\n\n  a',
 'an / youn\n  aback / pa sipriz (taken by surprise)\n  abandon / abandone',
 'kite\n  abase / abese (to lower in rank-office-prestige)',
 '~diminye (to lower physical)\n  abash / demoralize (to destroy the self-confidence of)\n  abbreviate / abreje\n  abdicate / abdike\n  abduct / kidnape',
 'anlve\n  abet / ensite (incite',
 'urge on',
 'instigate)',
 'eksite (excite)\n  abhor / deteste (loathe',
 'to regard with extreme   repugnance)',
 'rayi (hate)\n  abide / respekte (respect)\n  ability / abilite',
 'aptitud\n  abject / abjèk',
 'degoutan\n  able / kapab\n  abolish / aboli\n  abound / gen anpil',
 'abonde\n  about / apropo',
 'a\n  above / anlè',
 'soutèt\n  abrade (to srcape) / graje',
 'fwote (to rub)',
 'kòche\n  abreast / okouran\n  abridge / abreje\n  abroad / lòtbò\n  abrupt / avrip',
 'brid sou kou',
 'bripbrip\n  absence / absans\n  absolve / absoud\n  absorb / absòbe\n  abstain / abstrenn\n  absurd / absid\n  abuse / (n) abi',
 '(v) abize\n  academic / akade

In [ ]:
wordlist.to_csv("hatian_creole_dictionary_v3.csv")

In [ ]:
wordlist.to_json("hatian_creole_dictionary_v3.json")